In [ ]:
print(sys.path)
print(os.path.abspath('..'))

In [5]:
!python -W ignore train.py -jn split-training

2020-04-05 15:55:20.368160: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/apps/cuda10_0/nccl/2.4.7/lib:/opt/apps/cuda10_0/cudnn/7.6.2/lib64:/opt/apps/cuda/10.0/lib64:/opt/apps/intel17/python3/3.6.3/lib:/opt/intel/compilers_and_libraries_2017.4.196/linux/mpi/intel64/lib:/opt/intel/debugger_2017/libipt/intel64/lib:/opt/intel/debugger_2017/iga/lib:/opt/intel/compilers_and_libraries_2017.4.196/linux/daal/../tbb/lib/intel64_lin/gcc4.4:/opt/intel/compilers_and_libraries_2017.4.196/linux/daal/lib/intel64_lin:/opt/intel/compilers_and_libraries_2017.4.196/linux/tbb/lib/intel64/gcc4.7:/opt/intel/compilers_and_libraries_2017.4.196/linux/mkl/lib/intel64_lin:/opt/intel/compilers_and_libraries_2017.4.196/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2017.4.196/linux/ipp/lib/intel64:/opt/intel/compilers_and_l

2020-04-05 15:55:26.284967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0, 1, 2, 3
2020-04-05 15:55:26.285100: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2020-04-05 15:55:26.285115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 1 2 3 
2020-04-05 15:55:26.285125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N Y N N 
2020-04-05 15:55:26.285134: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 1:   Y N N N 
2020-04-05 15:55:26.285143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 2:   N N N Y 
2020-04-05 15:55:26.285152: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 3:   N N Y N 
2020-04-05 15:55:26.291918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/device:GPU:0 with 10481 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, com

**********Training from scratch**********
Training using single GPU or CPU..
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
axial (InputLayer)           [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 8)         80        
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 8)         32        
_________________________________________________________________
activation (Activation)      (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 8)         584       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 8)         32        
__________________________________________________

In [2]:
import os, sys
import datetime
import numpy as np
import random
import timeit
import argparse
import getpass

sys.path.append("..")
sys.path.append("../defacing")
from training.training import trainer
from helpers.utils import get_available_gpus
from distutils.dir_util import copy_tree
import tensorflow as tf

list_gpu = get_available_gpus()
n_gpu = len(list_gpu)
print("Available GPUs: ", list_gpu)

parser = argparse.ArgumentParser(description="Training DefacingNet")
parser.add_argument("--GPU", default="0", type=str, help="which GPU to use")
parser.add_argument(
    "-jn",
    "--job_name",
    required=True,
    type=str,
    help="The job name is required. All the training will be saved here.",
)

args = parser.parse_args()

t0 = timeit.default_timer()
# os.environ["CUDA_VISIBLE_DEVICES"] = args.GPU
print(
    "GPU Availability: ",
    tf.test.is_gpu_available(cuda_only=True, min_cuda_compute_capability=None),
)


Kfolds = 10
nfolds = list(range(1, Kfolds + 1))

for fold in nfolds:
    root_dir = "./Logs/" + args.job_name + "/train_test_fold_{}".format(fold)
    dir_path = "./Logs/" + args.job_name + "/train_test_fold_{}/csv/".format(fold)

    # currently a very hacky way of doing this -- will need to fix later
    from_dir = os.path.abspath(
        "./csv/faced_defaced/train_test_fold_{}/csv/".format(fold)
    )
    to_dir = dir_path
    copy_tree(from_dir, to_dir)

    train_csv_path = os.path.join(dir_path, "training.csv")
    valid_csv_path = os.path.join(dir_path, "validation.csv")

    # Model Path
    model_path = root_dir + "/" + args.job_name

    # create a path to where the model will be saved
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    if not os.path.exists(model_path):
        os.makedirs(model_path)

    # basic job info text file to identify jobs
    basic_job_info = os.path.join(os.path.abspath(root_dir), "job_info.txt")
    with open(basic_job_info, "w") as f:
        f.write("Jobname: %s\n" % args.job_name)
        f.write("Created on: %s\n" % str(datetime.datetime.now()))
        f.write("Created by: %s\n" % str(getpass.getuser()))
        f.write("Model store path: %s\n" % os.path.abspath(model_path))
        f.write(
            "GPU Availability: %s\n"
            % str(
                tf.test.is_gpu_available(
                    cuda_only=True, min_cuda_compute_capability=None
                )
            )
        )
        f.write("Available GPUs: %s\n" % (",".join(list_gpu)))

    train = trainer(
        train_csv_path,
        valid_csv_path,
        basic_job_info,
        model_path,
        image_size=32,
        batch_size=8,
        initial_epoch=0,
        nepochs=25,
        dropout=0.4,
        nclasses=2,
        nchannels=1,
        gpus=4,
    )
    train.train()

    elapsed = timeit.default_timer() - t0
    print("Time: {:.3f} min".format(elapsed / 60))
    del train


Available GPUs:  ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']


usage: ipykernel_launcher.py [-h] [--GPU GPU] -jn JOB_NAME
ipykernel_launcher.py: error: the following arguments are required: -jn/--job_name


SystemExit: 2

/opt/apps/intel17/python3/3.6.3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import matplotlib

matplotlib.use("Agg")

import os, sys
import glob
import random
import time
import imgaug
from imgaug import augmenters as iaa
import nibabel as nib
import SimpleITK as sitk
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
from six.moves import range

sys.path.append('..')
from helpers.utils import *
from skimage.restoration import denoise_wavelet


class DataGeneratoronFly(tf.keras.utils.Sequence):
    """
	"""

    def __init__(
        self,
        data_csv,
        nclasses=2,
        image_size=128,
        batch_size=32,
        nchannels=1,
        mode="Train",
        name=None,
        samples_per_epoch=None,
        transform=None,
    ):

        self.batch_size = batch_size
        self.image_size = image_size
        self.nchannels = nchannels
        self.nclasses = nclasses
        self.transform = transform
        self.name = name
        self.paths = []
        self.labels = []

        labels = pd.read_csv(data_csv)["Y"].values
        paths = pd.read_csv(data_csv)["X"].values

        index = np.arange(len(paths))
        np.random.shuffle(index)

        labels = labels[index]
        paths = paths[index]
        
        if mode.lower() in ["train", "valid", "test"]:
            self.mode = mode.lower()
        else:
            raise ValueError(
                "mode should be one among ['Train', 'Valid', 'Test'], given argument: {}".format(
                    mode
                )
            )
            
        if mode == "Train":
            minarr = [np.sum(labels == i) for i in range(nclasses)]
            mincount = np.min(minarr)
            for i in range(nclasses):
                self.paths.extend(paths[labels == i][:mincount])
                self.labels.extend(labels[labels == i][:mincount])

            self.paths = np.array(self.paths)
            self.labels = np.array(self.labels)

        elif mode == "Valid":
            self.paths = np.array(paths)
            self.labels = np.array(labels)

        assert len(np.unique(self.labels)) == nclasses
        self.len_arr = [sum(self.labels == arr) for arr in np.unique(self.labels)]

        index = np.arange(len(self.paths))
        np.random.shuffle(index)

        self.paths = self.paths[index]
        self.labels = self.labels[index]
        
        print(
            "============== paths: {}, labels: {} ================".format(
                len(self.paths), len(self.labels)
            )
        )
        
        if samples_per_epoch is None:
            if mode == "Train":
                self.samples_per_epoch = 4 * len(self.paths)
            else:
                self.samples_per_epoch = len(self.paths)
        else:
            self.samples_per_epoch = samples_per_epoch
            
        print(
            "============== Samples/Epoch: {} ================".format(
                self.samples_per_epoch
            )
        )

    def __len__(self):
        """
            Denotes the number of batches per epoch
        """
        return int(np.floor(self.samples_per_epoch / self.batch_size))

#     def __getitem__(self, index):
        
# #         print("getitem index: ", index)
#         # Generate indexes of the batch
#         X1, X2, X3, y = self.__data_generation(index)
#         if self.name == "combined":
#             return [X1, X2, X3], y
#         elif self.name == "axial":
# #             tf_dataset = tf.data.Dataset.from_tensor_slices((X1, y))
#             return X1, y
#         elif self.name == "coronal":
#             return X2, y
#         elif self.name == "sagittal":
#             return X3, y

    def _standardize_volume(self, volume, mask=None):
        """
			volume: volume which needs to be normalized
			mask: brain mask, only required if you prefer not to
				consider the effect of air in normalization
		"""
        if mask != None:
            volume = volume * mask

        mean = np.mean(volume[volume != 0])
        std = np.std(volume[volume != 0])

        return (volume - mean) / std

    def _normalize_volume(self, volume, mask=None, _type="MinMax"):
        """
			volume: volume which needs to be normalized
			mask: brain mask, only required if you prefer not to
				consider the effect of air in normalization
			_type: {'Max', 'MinMax', 'Sum'}
		"""
        if mask != None:
            volume = mask * volume

        min_vol = np.min(volume)
        max_vol = np.max(volume)
        sum_vol = np.sum(volume)

        if _type == "MinMax":
            return (volume - min_vol) / (max_vol - min_vol)
        elif _type == "Max":
            return volume / max_vol
        elif _type == "Sum":
            return volume / sum_vol
        else:
            raise ValueError(
                "Invalid _type, allowed values are: {}".format("Max, MinMax, Sum")
            )

    def _augmentation(self, volume):
        """
			Augmenters that are safe to apply to masks
			Some, such as Affine, have settings that make them unsafe, so always
			test your augmentation on masks
		"""
        volume_shape = volume.shape
        det = self.transform.to_deterministic()
        volume = det.augment_image(volume)

        assert volume.shape == volume_shape, "Augmentation shouldn't change volume size"
        return volume

    def _resizeVolume(self, volume):
        """
			resizes the original volume such that every patch is
			75% of original volume

			volume: numpy 3d tensor
		"""
        ratio = 1.0

        orig_size = (
            int(self.image_size / ratio),
            int(self.image_size / ratio),
            int(self.image_size / ratio),
        )
        resized_volume = resize_sitk(volume, orig_size)
        return resized_volume

    def _get_random_slices(self, volume):
        """
		"""
        dimensions = volume.shape
        img = np.zeros((dimensions[0], dimensions[1], 3))
        x = np.random.randint(dimensions[0] // 4, 3 * dimensions[0] // 4)
        z = np.random.randint(dimensions[1] // 4, 3 * dimensions[1] // 4)
        y = np.random.randint(dimensions[2] // 4, 3 * dimensions[2] // 4)
        slice_x = volume[x, :, :]
        slice_y = volume[:, y, :]
        slice_z = volume[:, :, z]

        return slice_x[..., None], slice_y[..., None], slice_z[..., None]

    def _center_align(self, volume):
        """
        """
        return volume

    def _axis_align(self, volume):
        """
        """
        return volume

    def __data_generation(self, index):
        """
        balanced data loader
        """
        X1, X2, X3 = [], [], []
        Y = []
        nclass_batch = self.batch_size // self.nclasses
        
#         print("Nclass batch: ", nclass_batch)
        
        for i in range(nclass_batch):
            for ii in np.unique(self.labels):
                # try:
                pid_path = self.paths[self.labels == ii][
                    int(index * nclass_batch + i) % self.len_arr[ii]
                ]
                label = ii  # np.eye(self.nclasses)[ii]

                volume, affine, size = load_vol(pid_path)
                volume = self._axis_align(volume)
                volume = self._center_align(volume)
                volume = self._resizeVolume(volume)
                volume = self._standardize_volume(volume)
                volume = self._normalize_volume(volume)

                if (self.mode.lower() == "train") and self.transform:
                    volume = self._augmentation(volume)

                ax, sg, co = self._get_random_slices(volume)

                if ax.shape == sg.shape == co.shape:
                    X1.append(ax)
                    X2.append(sg)
                    X3.append(co)
                    Y.append(label)
                # except:
                # 	continue

        X1, X2, X3, Y = np.array(X1), np.array(X2), np.array(X3), np.array(Y)
        
        
        index = np.arange(len(X1))
        np.random.shuffle(index)
        
        X1, X2, X3, Y = X1[index], X2[index], X3[index], Y[index]
        
#         print("X1.shape: ", X1.shape)
#         print(Y)
        
        return X1, X2, X3, Y
    
    

if __name__ == "__main__":

    dir_path = os.path.abspath("csv/faced_defaced/train_test_fold_1/csv/")

    csv_path = os.path.join(dir_path, "training.csv")
    
    print(dir_path, csv_path)
    
    augmentation = iaa.SomeOf(
        (0, 3),
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.5),
            iaa.Noop(),
            iaa.OneOf(
                [iaa.Affine(rotate=90), iaa.Affine(rotate=180), iaa.Affine(rotate=270)]
            ),
            # iaa.GaussianBlur(sigma=(0.0, 0.2)),
        ],
    )

    # Parameters
    train_transform_params = {
        "image_size": 128,
        "batch_size": 32,
        "nclasses": 2,
        "nchannels": 1,
        "name": "axial",
        "samples_per_epoch": None,
        "transform": augmentation,
    }

    valid_transform_params = {
        "image_size": 128,
        "batch_size": 32,
        "nclasses": 2,
        "nchannels": 1,
        "name": "axial",
        "samples_per_epoch": None,
        "transform": None,
    }

    # Generators
    training_generator = DataGeneratoronFly(data_csv=csv_path, **train_transform_params)
    
#     X, Y - (133*32, 128, 128, 1) (133*32)
    
    # print (training_generator.__len__())

#     validation_generator = DataGeneratoronFly(data_csv=csv_path, **valid_transform_params)
#     print(validation_generator.__len__())
    
#     print(training_generator[0][0].shape)
#     print(training_generator[0][1].shape)

    
#     train_dataset = tf.data.Dataset.from_tensor_slices(training_generator)
    
#     for X, y in training_generator:
#         print (X.shape, y.shape)
#         print (y[:4])
#         imshow(X[0,:,:,64, 0], X[1,:,:,64, 0], X[2,:,:,64, 0], X[3,:,:,64, 0])
        
        
#     for ep in range(5):
#         print ("============================")
#         for X, y in validation_generator:
#             print (X.shape, y.shape)
#             print (y[:4])
		# imshow(X[0,:,:,64, 0], X[1,:,:,64, 0], X[2,:,:,64, 0], X[3,:,:,64, 0])
        

#     import time
#     import matplotlib.pyplot as plt

#     start_time = time.time()
#     for i, (X, y) in enumerate(validation_generator):
#         elapsed_time = time.time() - start_time
#         start_time = time.time()
#         plt.subplot(1, 3, 1)
#         plt.imshow(X[0][0][:, :, 0])
#         plt.subplot(1, 3, 2)
#         plt.imshow(X[1][0][:, :, 0])
#         plt.subplot(1, 3, 3)
#         plt.imshow(X[2][0][:, :, 0])
#         plt.title(str(y[0]))
#         plt.savefig(str(i) + "_.png")
#         print(y, type(X))
#         print(X[0].shape, X[1].shape, X[2].shape)
#         print(i, "Elapsed Time", np.round(elapsed_time, decimals=2), "seconds")
#         pass


/work/06850/sbansal6/maverick2/mri-face-detector/defacing/examples/csv/faced_defaced/train_test_fold_1/csv /work/06850/sbansal6/maverick2/mri-face-detector/defacing/examples/csv/faced_defaced/train_test_fold_1/csv/training.csv
============== paths: 1068, labels: 1068 ================
============== Samples/Epoch: 4272 ================


NameError: name 'X' is not defined

In [8]:
import tensorflow as tf

import nobrainer
from nobrainer import dataset, volume

dir_path = os.path.abspath("csv/faced_defaced/train_test_fold_1/csv/")
csv_path = os.path.join(dir_path, "training.csv")
# print(dir_path, csv_path)


labels = pd.read_csv(csv_path)["Y"].values
paths = pd.read_csv(csv_path)["X"].values


# print(labels)

n_classes=2
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)


training_paths = zip(paths, labels)

print(training_paths)

ValueError: tensorflow>=2.0.0 must be installed but found version 1.14.0

In [5]:
import os, sys
sys.path.append('..')
from helpers.utils import load_vol, save_vol
from preprocessing.normalization import standardize_volume, normalize_volume
from preprocessing.conform import conform_data
import numpy as np
from glob import glob

orig_data_face   = '/work/01329/poldrack/data/mriqc-net/face/T1w'
orig_data_deface = '/work/01329/poldrack/data/mriqc-net/defaced'

save_data_face   = '/work/01329/poldrack/data/mriqc-net/face/derivatives/conformed'
save_data_deface = '/work/01329/poldrack/data/mriqc-net/defaced/derivatives/conformed'
os.makedirs(save_data_face, exist_ok=True)
os.makedirs(save_data_deface, exist_ok=True)

# os.listdir(save_data_face)
conform_size = (64, 64, 64)

def preprocess(pth):
    """
    """
    filename = pth.split('/')[-1]
    volume = conform_data(pth, conform_size)

    volume = normalize_volume(standardize_volume(volume))
    save_path = os.path.join(save_data_face, filename)

    newaffine = np.eye(4)
    newaffine[:3, 3] = -0.5 * (np.array(out_size) - 1)
    nii = nib.Nifti1Image(volume, newaffine, None)
    nii.to_filename(save_path)
    return save_path
